# 📊 MGMT 467 - Unit 2 Lab 2: Churn Modeling with BigQueryML + Feature Engineering
**Date:** 2025-10-16

In this lab you will:
- Connect to BigQuery from Colab
- Create features and labels
- Engineer new features from user behavior
- Train and evaluate logistic regression models
- Reflect on modeling assumptions and interpret results

In [1]:
# ✅ Authenticate and set up GCP project
from google.colab import auth
auth.authenticate_user()

project_id = "mgmt467-471819"  # <-- Replace with your actual project ID
!gcloud config set project $project_id

Updated property [core/project].


In [2]:
# ✅ Verify BigQuery access
%%bigquery --project $project_id
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-27,orion@tzannes.net


In [3]:
# ✅ Prepare base churn features
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;

Executing query with job ID: 367ac43c-d6f3-444f-83e8-ca64ce9b84ac
Query executing: 0.36s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: 367ac43c-d6f3-444f-83e8-ca64ce9b84ac



In [4]:
# ✅ Train base logistic regression model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `your_dataset.churn_features`;

Executing query with job ID: 52c0e0c6-2929-432a-9466-ba65aaa99aea
Query executing: 0.31s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: 52c0e0c6-2929-432a-9466-ba65aaa99aea



In [5]:
# ✅ Evaluate base model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);

Executing query with job ID: 63d1b2ac-4fdd-4e54-ba8d-f20bacabcfc9
Query executing: 0.41s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: 63d1b2ac-4fdd-4e54-ba8d-f20bacabcfc9



In [6]:
# ✅ Predict churn with base model
%%bigquery --project $project_id
SELECT
  user_id,
  predicted_churn_label,
  predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
                (SELECT * FROM `your_dataset.churn_features`));

Executing query with job ID: 6ae57f03-9306-429f-bac8-2e2c7c21020c
Query executing: 0.48s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: 6ae57f03-9306-429f-bac8-2e2c7c21020c




## 🛠️ Feature Engineering Section

We will now engineer new features to improve model performance:

- Bucket continuous variables
- Create interaction terms
- Add behavioral flags


In [7]:

# ✅ Create enhanced feature set
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `your_dataset.churn_features_enhanced` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
    ELSE 'high'
  END AS watch_time_bucket,
  avg_progress,
  num_sessions,
  CONCAT(plan_tier, '_', region) AS plan_region_combo,
  IF(total_minutes > 500, 1, 0) AS flag_binge,
  churn_label
FROM `your_dataset.churn_features`;


Executing query with job ID: eef899b3-2004-403f-ad03-13b2c31eb33a
Query executing: 0.42s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: eef899b3-2004-403f-ad03-13b2c31eb33a



In [8]:

# ✅ Train enhanced model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model_enhanced`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  watch_time_bucket,
  avg_rating,
  avg_progress,
  num_sessions,
  plan_region_combo,
  flag_binge,
  churn_label
FROM `your_dataset.churn_features_enhanced`;


Executing query with job ID: c5930adc-06a2-46f1-b885-fd8d2c8bc680
Query executing: 0.27s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: c5930adc-06a2-46f1-b885-fd8d2c8bc680



In [9]:

# ✅ Evaluate enhanced model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model_enhanced`);


Executing query with job ID: b325735c-facc-47c6-a00d-911bdef7b924
Query executing: 0.91s


ERROR:
 404 Not found: Dataset mgmt467-471819:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-471819:your_dataset was not found in location US

Location: US
Job ID: b325735c-facc-47c6-a00d-911bdef7b924




## 🤔 Chain-of-Thought Prompts: Feature Engineering

### 1. Why bucket continuous values like watch time?
- What patterns become clearer by using categories like "low", "medium", "high"?

### 2. What value do interaction terms (e.g., `plan_tier_region`) add?
- Could some plans behave differently in different regions?

### 3. What’s the purpose of binary flags like `flag_binge`?
- Can these capture unique behaviors not reflected in raw totals?

### 4. After evaluating the enhanced model:
- Which new features helped the most?
- Did any surprise you?

✍️ Write your responses in a text cell below or in a shared doc for discussion.





1.  **Bucketing watch time:** Grouping watch time into "low," "medium," and "high" can make it easier to see if there are clear differences in churn rates for these distinct groups. It helps simplify complex patterns.

2.  **Interaction terms:** Using `plan_tier_region` lets us see if a certain plan works differently in one region compared to another when it comes to keeping users. It captures unique combinations.

3.  **Binary flags:** A flag like `flag_binge` helps us identify specific behaviors, like really high watch time, that might be different from just having a lot of total minutes. It highlights unique actions.

4.  **After evaluating:** To see which new features were best, you'd need to check the results from evaluating the enhanced model (cell `423b6d00`). Look at the performance metrics and maybe the feature weights to see what made the biggest difference.